In [1]:
import pandas as pd
from pygris.data import get_census

About what 'poverty level' means:
https://www.census.gov/topics/income-poverty/poverty/about.html

TLDR: income thresholds are set that vary based on family size and composition.  does NOT vary geographically.  Updated for inflation.  Uses money income before taxes.

Status in PAST 12 MONTHS

In [2]:
api_key = '4489ebf8b64678c1d20d81cb44308726a18db5a2'

variables = [
    'B01003_001E',  #Total Population
    'B17001_001E',  # Total surveyed for poverty status
    'B17001_002E',  # Below Poverty Level
    'B17001_031E',  # At Or Above Poverty Level
]

# Get ACS data by state
acs_data = get_census(dataset='acs/acs5', 
                      year=2022, 
                      variables=variables, 
                      params = {
                          "for": "state:*"
                      })

acs_df = pd.DataFrame(acs_data)

acs_df.rename(columns={
    'B01003_001E': 'total_population',
    'B17001_001E': 'total_surveyed',
    'B17001_002E': 'below_poverty_level',
    'B17001_031E': 'above_poverty_level',
    'state': 'state_fips'
}, inplace=True)

# from us import states
# state_abbr = {state.fips: state.name for state in states.STATES}
# acs_df['state'] = acs_df['state_fips'].map(state_abbr)

# numeric_columns = [
#     'total_population',
#     'total_surveyed',
#     'below_poverty_level',
#     'above_poverty_level'
# ]

# # Convert columns to integers
# acs_df[numeric_columns] = acs_df[numeric_columns].astype(int)

numeric_columns = [col for col in acs_df.columns if col not in ['state_fips']]
acs_df[numeric_columns] = acs_df[numeric_columns].astype(int)

state_abbr = {
    '01': 'Alabama', '02': 'Alaska', '04': 'Arizona', '05': 'Arkansas', '06': 'California',
    '08': 'Colorado', '09': 'Connecticut', '10': 'Delaware', '11': 'District of Columbia',
    '12': 'Florida', '13': 'Georgia', '15': 'Hawaii', '16': 'Idaho', '17': 'Illinois',
    '18': 'Indiana', '19': 'Iowa', '20': 'Kansas', '21': 'Kentucky', '22': 'Louisiana',
    '23': 'Maine', '24': 'Maryland', '25': 'Massachusetts', '26': 'Michigan', '27': 'Minnesota',
    '28': 'Mississippi', '29': 'Missouri', '30': 'Montana', '31': 'Nebraska', '32': 'Nevada',
    '33': 'New Hampshire', '34': 'New Jersey', '35': 'New Mexico', '36': 'New York',
    '37': 'North Carolina', '38': 'North Dakota', '39': 'Ohio', '40': 'Oklahoma', '41': 'Oregon',
    '42': 'Pennsylvania', '44': 'Rhode Island', '45': 'South Carolina', '46': 'South Dakota',
    '47': 'Tennessee', '48': 'Texas', '49': 'Utah', '50': 'Vermont', '51': 'Virginia',
    '53': 'Washington', '54': 'West Virginia', '55': 'Wisconsin', '56': 'Wyoming'
}

# Map state FIPS codes to state names
acs_df['state'] = acs_df['state_fips'].map(state_abbr)

# Move the 'state' column to the beginning
cols = ['state'] + [col for col in acs_df.columns if col != 'state']
acs_df = acs_df[cols]

acs_df.head()

,state,total_population,total_surveyed,below_poverty_level,above_poverty_level,state_fips
0,Alabama,5028092,4890427,768897,4121530,01
1,Alaska,734821,717293,75227,642066,02
2,Arizona,7172282,7017776,916876,6100900,04
3,Arkansas,3018669,2931377,475729,2455648,05
4,California,39356104,38643585,4685272,33958313,06


In [3]:
acs_df['pct_below'] = acs_df['below_poverty_level'] / acs_df['total_surveyed']

In [4]:
# Reorder columns
cols = ['state', 'total_population', 'total_surveyed', 
        'below_poverty_level', 'pct_below','above_poverty_level', 'state_fips']

# Select columns in the desired order
acs_df = acs_df[cols]

acs_df.head()

,state,total_population,total_surveyed,below_poverty_level,pct_below,above_poverty_level,state_fips
0,Alabama,5028092,4890427,768897,0.157225,4121530,01
1,Alaska,734821,717293,75227,0.104876,642066,02
2,Arizona,7172282,7017776,916876,0.130651,6100900,04
3,Arkansas,3018669,2931377,475729,0.162289,2455648,05
4,California,39356104,38643585,4685272,0.121243,33958313,06


In [5]:
csv_file_path = 'poverty_data.csv'
acs_df.to_csv(csv_file_path, index=False)